In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
import astropy.units as units
from lsst.rsp import get_tap_service, retrieve_query
from lsst.utils.plotting import set_rubin_plotstyle, stars_cmap
from matplotlib import gridspec
import matplotlib.patheffects as pathEffects
import matplotlib.ticker as ticker

In [ ]:
service = get_tap_service("tap")

In [ ]:
query = "SELECT coord_ra, coord_dec, "\
        "scisql_nanojanskyToAbMag(u_psfFlux) as umag, scisql_nanojanskyToAbMagSigma(u_psfFlux, u_psfFluxErr) as umagErr, u_psfFlux_flag, "\
        "scisql_nanojanskyToAbMag(g_psfFlux) as gmag, scisql_nanojanskyToAbMagSigma(g_psfFlux, g_psfFluxErr) as gmagErr, g_psfFlux_flag, "\
        "scisql_nanojanskyToAbMag(r_psfFlux) as rmag, scisql_nanojanskyToAbMagSigma(r_psfFlux, r_psfFluxErr) as rmagErr, r_psfFlux_flag, "\
        "scisql_nanojanskyToAbMag(i_psfFlux) as imag, scisql_nanojanskyToAbMagSigma(i_psfFlux, i_psfFluxErr) as imagErr, i_psfFlux_flag, "\
        "scisql_nanojanskyToAbMag(z_psfFlux) as zmag, scisql_nanojanskyToAbMagSigma(z_psfFlux, z_psfFluxErr) as zmagErr, z_psfFlux_flag, "\
        "scisql_nanojanskyToAbMag(y_psfFlux) as ymag, scisql_nanojanskyToAbMagSigma(y_psfFlux, y_psfFluxErr) as ymagErr, y_psfFlux_flag "\
        "FROM dp1.Object "\
        "WHERE refExtendedness < 0.5 AND scisql_nanojanskyToAbMagSigma(i_psfFlux, i_psfFluxErr) < 0.2"
job = service.submit_job(query)
print('Job URL is', job.url)
print('Job phase is', job.phase)
job.run()

In [ ]:
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
results = job.fetch_result().to_table()
print(len(results))

In [ ]:
set_rubin_plotstyle()

In [ ]:
cmap = stars_cmap()


In [ ]:
fig = plt.figure(figsize=(8, 8))
gs = gridspec.GridSpec(16, 16, hspace=0.01, wspace=0.01)

# Define axes
ax_main = fig.add_subplot(gs[4:, :])
cax = fig.add_subplot(gs[4:, 15])  # colorbar axis
use, = np.where((results["imagErr"] < 0.02) & ~results["g_psfFlux_flag"] & ~results["r_psfFlux_flag"] & ~results["i_psfFlux_flag"])
print("Number of stars in plot: ", use.size)
hb = ax_main.hexbin(results["gmag"][use] - results["rmag"][use], results["rmag"][use] - results["imag"][use], bins="log", extent=[-0.2, 2, -0.2, 2.0], gridsize=400, cmap=stars_cmap())
ax_main.set_xlabel("g - r (mag)")
ax_main.set_ylabel("r - i (mag)")
cb = fig.colorbar(hb, cax=cax, format="%.0f")
label="Points Per Bin"
text = cax.text(0.5, 0.5, label, color="k", 
                rotation="vertical", 
                transform=cax.transAxes, 
                ha="center",
                va="center", 
                fontsize=12)
text.set_path_effects([pathEffects.Stroke(linewidth=3, foreground="w"), pathEffects.Normal()])
plt.savefig(
    "dp1_stellar_locus_gri.pdf", 
    bbox_inches="tight",  # Trim whitespace around the figure
    transparent=True,     # Transparent background
    format="pdf",         # Explicit format specification
)
plt.show()


In [ ]:
fig = plt.figure(figsize=(8, 8))
gs = gridspec.GridSpec(16, 16, hspace=0.01, wspace=0.01)

# Define axes
ax_main = fig.add_subplot(gs[4:, :])
cax = fig.add_subplot(gs[4:, 15])  # colorbar axis
use, = np.where((results["imagErr"] < 0.02) & ~results["r_psfFlux_flag"] & ~results["i_psfFlux_flag"] & ~results["z_psfFlux_flag"])
print("Number of stars in plot: ", use.size)
hb = ax_main.hexbin(results["rmag"][use] - results["imag"][use], results["imag"][use] - results["zmag"][use], bins="log", extent=[-0.2, 2, -0.2, 1.0], gridsize=400, cmap=stars_cmap())
ax_main.set_xlabel("r - i (mag)")
ax_main.set_ylabel("i - z (mag)")
cb = fig.colorbar(hb, cax=cax, format="%.0f")
label="Points Per Bin"
text = cax.text(0.5, 0.5, label, color="k", 
                rotation="vertical", 
                transform=cax.transAxes, 
                ha="center",
                va="center", 
                fontsize=12)
text.set_path_effects([pathEffects.Stroke(linewidth=3, foreground="w"), pathEffects.Normal()])
plt.savefig(
    "dp1_stellar_locus_riz.pdf", 
    bbox_inches="tight",  # Trim whitespace around the figure
    transparent=True,     # Transparent background
    format="pdf",         # Explicit format specification
)
plt.show()


In [ ]:
def custom_formatter(x, pos):
    return f"{x:.0f}"

fig = plt.figure(figsize=(8, 8))
gs = gridspec.GridSpec(16, 16, hspace=0.01, wspace=0.01)

# Define axes
ax_main = fig.add_subplot(gs[4:, :])
cax = fig.add_subplot(gs[4:, 15])  # colorbar axis
use, = np.where((results["umagErr"] < 0.05) & ~results["u_psfFlux_flag"] & ~results["g_psfFlux_flag"] & ~results["r_psfFlux_flag"])
print("Number of stars in plot: ", use.size)
hb = ax_main.hexbin(results["umag"][use] - results["gmag"][use], results["gmag"][use] - results["rmag"][use], bins="log", extent=[0.5, 2.7, -0.2, 1.5], gridsize=400, cmap=stars_cmap())
ax_main.set_xlabel("u - g (mag)")
ax_main.set_ylabel("g - r (mag)")
cb = fig.colorbar(hb, cax=cax, format=ticker.FuncFormatter(custom_formatter))
label="Points Per Bin"
text = cax.text(0.5, 0.5, label, color="k", 
                rotation="vertical", 
                transform=cax.transAxes, 
                ha="center",
                va="center", 
                fontsize=12)
text.set_path_effects([pathEffects.Stroke(linewidth=3, foreground="w"), pathEffects.Normal()])
plt.savefig(
    "dp1_stellar_locus_ugr.pdf", 
    bbox_inches="tight",  # Trim whitespace around the figure
    transparent=True,     # Transparent background
    format="pdf",         # Explicit format specification
)
plt.show()
